In [103]:
import pandas as pd
import numpy as np
import csv
import json
import datetime as dt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [104]:
def json2csv(json_name, csv_name, key):
    with open(json_name) as json_file:
        data = json.load(json_file)
    transaction_data = data[key]
    data_file = open(csv_name, 'w') 
    csv_writer = csv.writer(data_file)
    count = 0
    for tran in transaction_data:
        if count == 0: 
            # Writing headers of CSV file 
            header = tran.keys()
            csv_writer.writerow(header) 
            count += 1
        # Writing data of CSV file 
        csv_writer.writerow(tran.values())
    data_file.close() 

In [105]:
json2csv('../web-app/server/src/allTransactions.json', 'allTransactions.csv', 'data')

In [106]:
df = pd.read_csv('allTransactions.csv')

In [128]:
df["month"] = ""

In [134]:
for i in range(df.shape[0]):
    df["month"][i] = df["valueDate"][i][5:7]

/home/fish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [135]:
#50/30/20 rule:
'''
(After all taxes) (per month)
50% - essentials
30% - wants
20% - savings

Income = all credit
'''

'\n(After all taxes) (per month)\n50% - essentials\n30% - wants\n20% - savings\n\nIncome = all credit\n'

In [143]:
df = df[df["month"] == "07"]
df = df.reset_index()

In [144]:
categories_list = set(df["category"].tolist())

In [145]:
categories_list

{'Bills',
 'Cash Withdrawal',
 'Credit Card',
 'Entertainment',
 'Foods and Drinks',
 'Fuel',
 'Groceries',
 'Insurance',
 'Investment',
 'Loan (EMI)',
 'Other',
 'Rent',
 'Shopping'}

In [146]:
income_list = ["Other"] #all credit
ess_list = ["Bills", "Cash Withdrawal", "Groceries", "Rent", "Loan (EMI)", "Personal Care", "Health", "Insurance", "Fuel", ]
wants_list = ["Credit Card", "Entertainment", "Food and Drinks", "Travel", "Transfer"]
savings_list = ["Investment"]
#len(income)+len(ess_list)+len(wants)

In [147]:
income_amnt = 0
ess_amnt = 0
wants_amnt = 0
savings_amnt = 0
for i in range(df.shape[0]):
    c = df["category"][i]
    amt = int(df["amount"][i])
    if c in income_list:
        income_amnt += amt
    elif c in ess_list:
        ess_amnt += amt
    elif c in wants_list:
        wants_amnt += amt
    elif c in savings_list:
        savings_amnt += amt
        

In [148]:
(income_amnt, ess_amnt, wants_amnt, savings_amnt)

(175005, 48054, 16791, 29000)

In [149]:
ess_base = income_amnt*.50
wants_base = income_amnt*.30
savings_base = income_amnt *.20

In [150]:
sav = income_amnt - (ess_amnt + wants_amnt + savings_amnt)

In [151]:
savings_amnt += sav

In [152]:
savings_amnt

110160

In [153]:
split = ["Income", "Needs", "Wants", "Savings"] 

In [154]:
base = [int(income_amnt), int(ess_base), int(wants_base), int(savings_base)]
current = [income_amnt, ess_amnt, wants_amnt, savings_amnt]

In [155]:
comparison_df = pd.DataFrame({"split": split, "base":base, "current":current})

In [156]:
comparison_df["msg"]=""

In [157]:
for i in range(1,3):
    name = comparison_df["split"][i]
    bas = comparison_df["base"][i]
    cur = comparison_df["current"][i]
    if bas > cur:
        msg = "You can spend "+ "₹ "+ str(bas-cur) + " more on " + name+"!"
    elif bas<cur:
        msg = "You are spending more on "+name+". You should spend around ₹ "+str(bas)+ " on "+name+"!"
    else:
        msg = "You are managing your " +name +" properly"
    comparison_df["msg"][i] = msg

/home/fish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [160]:
#Msg for income
comparison_df["msg"][0] = "This is your this month's income!"
#Msg for savings
bas = comparison_df["base"][3]
cur = comparison_df["current"][3]
if bas == cur:
    comparison_df["msg"][3] = "You are managing your savings very well!"
elif bas>cur:
    comparison_df["msg"][3] = "You should save more money for a stable future. Trying saving atleast ₹ " + bas + " next month!"
elif cur>bas:
    comparison_df["msg"][3] = "You are managing your savings very well!"

/home/fish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/fish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [162]:
comparison_df

,split,base,current,msg
0,Income,175005,175005,This is your this month's income!
1,Needs,87502,48054,You can spend ₹ 39448 more on Needs!
2,Wants,52501,16791,You can spend ₹ 35710 more on Wants!
3,Savings,35001,110160,You are managing your savings very well!
